# Santander Dev Week 2023

In [11]:
# Utilize sua própria URL se quiser :)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'
# Coloque na variavel openai_api_key sua key
openai_api_key = ''

### Imports and Frameworks

In [12]:
import pandas as pd
import requests
import json
import openai

## Extract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente

In [13]:
df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[15, 6, 10]


In [14]:
def get_user(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 15,
    "name": "Pyterson",
    "account": {
      "id": 22,
      "number": "0001-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 15,
      "number": "1234 1234 1234 1111",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "id": 51,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Invista hoje, colha amanh\u00e3! #FuturoBrilhante"
      },
      {
        "id": 132,
        "icon": "\ud83e\udd11",
        "description": null
      },
      {
        "id": 140,
        "icon": "\ud83e\udd11",
        "description": null
      },
      {
        "id": 148,
        "icon": "\ud83e\udd11",
        "description": "Pyterson, O futuro \u00e9 hoje: invista para colher amanh\u00e3."
      },
      {
        "id": 156,
        "icon": "\ud83e\udd11",
        "description": "Pyterson, Invista hoje, desfrute no futuro."


## Transform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de Marketing personalizada para cada usuário

In [15]:
openai.api_key = openai_api_key

def generate_ai_news(user):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system", 
                "content": "Você é um especialista em Marketing Bancário"
            },
            {
                "role": "user", 
                "content": f"Crie uma mensagem para {user['name']}, sobre a importância dos investimentos (máximo de 100 caracteres)"
            }
        ]
    )
    return completion.choices[0].message.content.strip('\"')

for user in users:
    news = generate_ai_news(user)
    print(news) 

Invista hoje, colha amanhã! #SucessoFinanceiro
Invista hoje, garanta seu futuro! 💰
Invista no seu futuro, Laura!


## Load

Atualize a lista de 'news' de cada usuário na API com a nova mensagem gerada

In [16]:
def update_user(user):
    response = requests.put(f'{sdw2023_api_url}/users/{user["id"]}', json=user)
    return True if response.status_code == 200 else False

for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated {success}!")

User Pyterson updated True!
User Pep updated True!
User laura updated True!
